Run Cell below

In [2]:
"""Calibrate LWR with Gipps"""
from scipy.optimize import minimize
import pandas as pd
import numpy as np
import os
from flow.core.macroscopic import LWR
from flow.core.macroscopic.lwr import PARAMS as LWR_PARAMS
from flow.core.macroscopic.utils import run
from flow.core.macroscopic.Calibration.aggregate_micro_to_macro import agg_func
params = LWR_PARAMS.copy()

dir = "./Gipps/"

L = 2000
dx = 10
params['length'] = L
params['dx'] = dx
params['v_max_max'] = 100
params['rho_max_max'] = 300
params['total_time'] = 300
params["boundary_conditions"] = "loop"

Avg_pos = pd.DataFrame()
Avg_speed = pd.DataFrame()
file_list = os.listdir(dir)
for file in file_list:
    if "pos" in file:
        T = pd.read_csv(dir + file)
        Avg_pos = pd.concat([Avg_pos, T])
    if "speed" in file:
        S = pd.read_csv(dir + file)
        Avg_speed = pd.concat([Avg_speed, S])

## mean positions from csv
Pos = Avg_pos.groupby("time").mean()
## mean velocities from csv
Speeds = Avg_speed.groupby("time").mean()
display(Pos.head())
display(Speeds.head())


# Single Run below

In [3]:
## get densities amd velocities -> at specific time step t and t+1
t1 = 20
row = np.array(Pos.loc[t1])
speed = np.array(Speeds.loc[t1])
rho_init, vel_init = agg_func(row, speed, L, dx)
params['initial_conditions'] = rho_init

t2 = 21
row = np.array(Pos.iloc[t2])
speed = np.array(Speeds.iloc[t2])
rho_final, vel_final = agg_func(row, speed, L, dx)
##################################################################
display(Pos.iloc[[t1]])
display(Pos.iloc[[t2]])

In [11]:
#single run
params['rho_max'] =1
params['v_max'] = 11
params['CFL'] = 0.95
params['initial_conditions'] = rho_init
params['dt'] = 0.1
env = LWR(params)
obs = env.reset()
obs, _, _, _ = env.step(rl_actions = params['v_max']) # one time step
obs[:int(len(obs)/2)]


array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.17258184, 0.24626816, 0.2675676 , 0.2509824 , 0.28986018,
       0.25211482, 0.25      , 0.493125  , 0.5       , 0.5       ,
       0.5       , 0.5       , 0.5       , 0.256875  , 0.25      ,
       0.25      , 0.25      , 0.25      , 0.25      , 0.25      ,
       0.25      , 0.25      , 0.25      , 0.25      , 0.25      ,
       0.49327743, 0.53707257, 0.5       , 0.46292743, 0.25672257,
       0.25      , 0.493125  , 0.5       , 0.256875  , 0.25      ,
       0.25      , 0.25      , 0.25      , 0.25      , 0.26493475,
       0.25084025, 0.23994823, 0.24940177, 0.586275  , 0.5       ,
       0.1650675 , 0.0141575 , 0.        , 0.        , 0.        ,
       0.        , 0.229375  , 0.25      , 0.25      , 0.493125  ,
       0.256875  , 0.020625  , 0.        , 0.22663543, 0.24983

In [43]:
## run lwr model one step and get densities and velocities
def objective(x):
    
    params['rho_max'] = x[0]
    params['v_max'] = x[1]
    params['CFL'] = x[2]
    #params['dt'] = x[2] * dx / x[1]
    params['dt'] = x[3]
    
    env = LWR(params)
    obs = env.reset()
    obs, _, _, _ = env.step(rl_actions = params['v_max']) # one time step
    lwr_density_pred = obs[:int(obs.shape[0]/2)]
    lwr_speeds_pred = obs[int(obs.shape[0]/2):]

    #loss
    loss = np.mean(np.abs(rho_final - lwr_density_pred))
    #loss2 = np.mean(np.abs(vel_final - lwr_speeds_pred))
    return loss

#set contraints
def constraint1(x):
    return (x[2] * dx / x[1]) - x[3]
con1 = {"type": "ineq", "fun":constraint1}

#set inital values and bounds
x0 = [1, 11, 0.95, 0.1]
bnds = ((0.0,1.0),(0.0,30),(0.0,0.99),(0,0.5))


# solve single run
sol = minimize(objective,x0, method="SLSQP", bounds=bnds, constraints=con1)
print("===================")
print( "Initial Values: => " + str(x0))
print("Initial Objective: =>"  +str(objective(x0)))
print("===================")
print("Optimal values: =>" + str(sol.x))
print("Optimal min_value: =>"+ str(sol.fun))

Initial Values: => [1, 11, 0.95, 0.1]
Initial Objective: =>0.13830729427250013
Optimal values: =>[ 1.         11.00378865  0.95        0.5       ]
Optimal min_value: =>0.13807037155605983


In [44]:
print("================Metrics=================")
print(sol)

================Metrics=================
     fun: 0.13807037155605983
     jac: array([-2.99667753e-03, -2.63433903e-05,  0.00000000e+00, -5.79757616e-04])
 message: 'Optimization terminated successfully.'
    nfev: 42
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([ 1.        , 11.00378865,  0.95      ,  0.5       ])


## Tensorflow setup w/Aboudy

In [16]:
import tensorflow as tf
# params['rho_max'] =1
# params['v_max'] = 11
# params['CFL'] = 0.95
# params['initial_conditions'] = rho_init
# params['dt'] = 0.1


def build_model(in_val):
    params = tf.Variable([1, 11], dtype=tf.float32)
    rho_max = params[0]
    v_max = params[1]
    CFL = 0.95
    
    env = LWR(params)
    env.reset()
    obs = env._step(in_val, v_max, rho_max, CFL) # one time step

    return obs

input_ph = tf.placeholder(tf.float32, shape=(None, 100))

y_pred = build_model(input_ph)

TypeError: must be str, not int

In [ ]:
x = data[:-1]
y_true = data[1:]
y_true_ph = tf.placeholder(tf.float32, shape(None,None))

loss = tf.reduce_mean(tf.square(y_pred - y_val))

# optimizer -> computes of gradients and updates our variables of interest (beta in our case)
optimizer = tf.train.AdamOptimizer(0.1).minimize(loss) # propagates backwards in our

####### run
init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init) # assign values to variables. Run to execute node.
    feed_dict = {input_ph: np.array(x), y_true_ph: np.array(y_true)} # static real data

    for _ in np.arange(30):
        #current beta
        print(beta_val.eval())
        #solve. Run to execute node.
        loss_val, _ = session.run([loss, optimizer], feed_dict)
        # current solve
        print(loss_val)

    # Plot results. Run to execute node.
    plt.scatter(x, y)
    plt.plot(x,session.run(y_pred_val,{x_val:x}), "-r")
    plt.show()

# Batch Run Below

In [8]:
def sample_t(Pos, size=10):
    return np.random.choice(Pos.index[:-1],size,replace=False)
index = sample_t(Pos, size=3)
next_index = index+1
index,next_index
matrix_initial = Pos.iloc[index]
matrix_final = Pos.iloc[next_index]
# display(matrix_initial.head())
# display(matrix_final.head())
# get densities amd velocities -> at specific time step t and t+1
A = matrix_initial.apply(lambda row : agg_func(row, row, L, dx)[0], axis = 1)
B = matrix_final.apply(lambda row : agg_func(row, row, L, dx)[0], axis = 1)

In [6]:
#Batch run
params['rho_max'] =1
params['v_max'] = 11
params['CFL'] = 0.95
params['initial_conditions'] = np.vstack(A)
params['dt'] = 0.1
params["boundary_conditions"] = "loop"
env = LWR(params)
obs = env.reset()
obs, _, _, _ = env.step(rl_actions = params['v_max']) # one time step


array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
      

In [9]:
# Batch run 2:
rho_matrix = A.values
def objective_batch(x):
    
    params['rho_max'] = x[0]
    params['v_max'] = x[1]
    params['CFL'] = x[2]
    #params['dt'] = x[2] * dx / x[1]
    params['dt'] = x[3]
    lwr_pred =[]
    
    params['initial_conditions'] = np.vstack(A)
    env = LWR(params)
    obs = env.reset()
    obs, _, _, _ = env.step(rl_actions = params['v_max']) # one time step

    lwr_pred = obs[:int(len(obs)/2)]

    #loss
    loss = np.mean(np.abs(np.vstack(B) - lwr_pred))

    return loss

def constraint1(x):
    return (x[2] * dx / x[1]) - x[3]
con1 = {"type": "ineq", "fun":constraint1}

#set inital values and bounds
x0_ = [1, 11, 0.95, 0.1]
bnds = ((0.0,1.0),(0.0,30),(0.0,0.99),(0,0.5))

# solve single run
sol2 = minimize(objective_batch,x0_, method="SLSQP", bounds=bnds, constraints=con1)
print("===================")
print( "Initial Values: => " + str(x0_))
print("Initial Objective: =>"  +str(objective_batch(x0_)))
print("===================")
print("Optimal values: =>" + str(sol2.x))
print("Optimal min_value: =>"+ str(sol2.fun))

Initial Values: => [1, 11, 0.95, 0.1]
Initial Objective: =>0.01610539191483332
Optimal values: =>[9.88857530e-01 1.09992027e+01 9.50000000e-01 4.16333634e-17]
Optimal min_value: =>0.01497549999999985


In [10]:
print("================Metrics=================")
print(sol2)

================Metrics=================
     fun: 0.01497549999999985
     jac: array([0.        , 0.        , 0.        , 0.00708009])
 message: 'Optimization terminated successfully.'
    nfev: 24
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([9.88857530e-01, 1.09992027e+01, 9.50000000e-01, 4.16333634e-17])


# Example Tensor FLOW

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

part_1_data = pd.read_csv("https://github.com/DS-100/fa19/raw/gh-pages/assets/datasets/hw7_data.csv", index_col=0)

x = part_1_data['x']
y = part_1_data['y']

# x
x_val = tf.placeholder(tf.float64, shape=(None,))
# y,
y_val = tf.placeholder(tf.float64, shape=(None,))
# beta -> this is the guys that gets updated magically
beta_val = tf.Variable(np.array([0.0, 0.0]))
# y_pred
y_pred_val = tf.multiply(x_val, beta_val[0]) + tf.sin(tf.multiply(x_val, beta_val[1]))

# loss
loss = tf.reduce_mean(tf.square(y_pred_val - y_val))

# optimizer -> computes of gradients and updates our variables of interest (beta in our case)
optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss) # propagates backwards in our

####### run
init = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init) # assign values to variables. Run to execute node.
    feed_dict = {x_val: np.array(x), y_val: np.array(y)} # static real data

    for _ in np.arange(30):
        #current beta
        print(beta_val.eval())
        #solve. Run to execute node.
        loss_val, _ = session.run([loss, optimizer], feed_dict)
        # current solve
        print(loss_val)

    # Plot results. Run to execute node.
    plt.scatter(x, y)
    plt.plot(x,session.run(y_pred_val,{x_val:x}), "-r")
    plt.show()
